In [108]:
import pandas as pd
import ast

### Dataframe y función para  PlayTimeGenre(Endpoint 1)
Se debe crear una funcion que, al ingresar el género, debe devolver el año con mas horas jugadas para dicho género.
Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

Para ello necesitamos extraer del archivo "mergueFinal" las siguientes columnas:
 
genres, 
release_date, y
playtime_forever

Cargamos los dos datasets mergueFina.parquet:

In [109]:
rutamerge = r'C:\Users\Coder\Documents\PI_ML_OPS\Datos\mergefinal.parquet'
play_time_genre = pd.read_parquet(rutamerge)

In [110]:
play_time_genre 

,user_id,title,developer,genres,release_date,price,playtime_forever,posted,item_id,recommend,sentiment_analysis
0,76561197970982479,Counter-Strike,Valve,['Action'],2000.0,9.99,6,2011.0,1250,True,2
1,js41637,Counter-Strike,Valve,['Action'],2000.0,9.99,0,2014.0,251610,True,2
2,WeiEDKrSat,Counter-Strike,Valve,['Action'],2000.0,9.99,328,2013.0,4000,True,1
3,jarrodtrim,Counter-Strike,Valve,['Action'],2000.0,9.99,337,2012.0,440,True,2
4,UTNerd24,Counter-Strike,Valve,['Action'],2000.0,9.99,5,2014.0,244210,True,1
...,...,...,...,...,...,...,...,...,...,...,...
17623,celebrexISGO,Insurgency,New World Interactive,"['Action', 'Indie', 'Strategy']",2014.0,9.99,0,2015.0,730,True,1
17624,BBiiiirr,Real Horror Stories Ultimate Edition,GameORE,['Indie'],2014.0,1.99,0,2015.0,440,True,1
17625,Hahneex,Infinity Wars: Animated Trading Card Game,Lightmare Studios,"['Free to Play', 'Indie', 'Massively Multiplay...",2014.0,0.00,0,2015.0,730,True,0
17626,76561198222628548,Dungeon Defenders II,Trendy Entertainment,"['Action', 'Free to Play', 'Indie', 'RPG', 'St...",2017.0,0.00,1222,2015.0,370240,False,0


In [111]:
#Eliminamos las columna innecesarias:

columnas_a_eliminar = ['user_id', 'title', 'developer', 'price', 'posted', 'item_id', 'recommend', 'sentiment_analysis']
play_time_genre.drop(columnas_a_eliminar, axis=1, inplace=True)

In [112]:
play_time_genre.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17628 entries, 0 to 17627
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   genres            17628 non-null  object 
 1   release_date      17628 non-null  float64
 2   playtime_forever  17628 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 413.3+ KB


In [113]:
# Cambiamos el tipo de dato de la columna release_date a entero
play_time_genre['release_date'] = play_time_genre['release_date'].astype(int)

In [114]:
play_time_genre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17628 entries, 0 to 17627
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   genres            17628 non-null  object
 1   release_date      17628 non-null  int32 
 2   playtime_forever  17628 non-null  int64 
dtypes: int32(1), int64(1), object(1)
memory usage: 344.4+ KB


Desanidamos la columna genres, asignando un nuevo registro para cada genero:

In [115]:

play_time_genre['genres'] = play_time_genre['genres'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else x)
play_time_genre = play_time_genre.explode('genres')
play_time_genre

,genres,release_date,playtime_forever
0,Action,2000,6
1,Action,2000,0
2,Action,2000,328
3,Action,2000,337
4,Action,2000,5
...,...,...,...
17626,Indie,2017,1222
17626,RPG,2017,1222
17626,Strategy,2017,1222
17627,Action,2013,69


Transformamos todos los registros de genres a lowercase para evitar diferencias entre minúscula y mayúscula:

In [116]:

play_time_genre['genres'] = play_time_genre['genres'].str.lower()


In [117]:
play_time_genre.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25906 entries, 0 to 17627
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   genres            25906 non-null  object
 1   release_date      25906 non-null  int32 
 2   playtime_forever  25906 non-null  int64 
dtypes: int32(1), int64(1), object(1)
memory usage: 708.4+ KB


Guardamos el dataframe en CSV:

In [118]:
play_time_genre.to_csv(r'C:\Users\Coder\Documents\PI_ML_OPS\Datos\play_time_genre.csv', index=False, errors='replace', encoding='utf-8')

FUNCION END POINT(1): PlayTimeGenre

In [119]:
def PlayTimeGenre(genero: str):
    # Se filtra el DataFrame por el género indicado
    genre_df = play_time_genre[play_time_genre['genres'] == genero]

    if genre_df.empty:
        return f"No hay datos para el género {genero}."

    # Se realiza una agrupación por año y se suman las horas playtime
    grouped_df = genre_df.groupby('release_date')['playtime_forever'].sum()

    # Se realiza una búsqueda para el año con más horas jugadas
    max_year = grouped_df.idxmax()

    return {f"Año de lanzamiento con más horas jugadas para el género {genero}": max_year}



In [120]:
# Comprobamos la función:
genero_entrada = "indie"
resultado = PlayTimeGenre(genero_entrada)
print(resultado)

{'Año de lanzamiento con más horas jugadas para el género indie': 2006}


_______________________________________________________________________________________________________________

### Dataframe y función para UserForGenre (Endpoint 2)
Se debe crar una función que, al ingresar un género, retorne el usuario que acumule más horas jugadas para el género y una lista de la acumulación de horas jugadas por año.

Pare ello necesitamos extraer del archivo "mergueFinal" las siguientes columnas:

user_id,
genres, 
release_date, y
playtime_forever

Cargamos los dos datasets mergueFina.parquet:

In [121]:
rutamerge = r'C:\Users\Coder\Documents\PI_ML_OPS\Datos\mergefinal.parquet'
df_user_genre = pd.read_parquet(rutamerge)

In [122]:
df_user_genre

,user_id,title,developer,genres,release_date,price,playtime_forever,posted,item_id,recommend,sentiment_analysis
0,76561197970982479,Counter-Strike,Valve,['Action'],2000.0,9.99,6,2011.0,1250,True,2
1,js41637,Counter-Strike,Valve,['Action'],2000.0,9.99,0,2014.0,251610,True,2
2,WeiEDKrSat,Counter-Strike,Valve,['Action'],2000.0,9.99,328,2013.0,4000,True,1
3,jarrodtrim,Counter-Strike,Valve,['Action'],2000.0,9.99,337,2012.0,440,True,2
4,UTNerd24,Counter-Strike,Valve,['Action'],2000.0,9.99,5,2014.0,244210,True,1
...,...,...,...,...,...,...,...,...,...,...,...
17623,celebrexISGO,Insurgency,New World Interactive,"['Action', 'Indie', 'Strategy']",2014.0,9.99,0,2015.0,730,True,1
17624,BBiiiirr,Real Horror Stories Ultimate Edition,GameORE,['Indie'],2014.0,1.99,0,2015.0,440,True,1
17625,Hahneex,Infinity Wars: Animated Trading Card Game,Lightmare Studios,"['Free to Play', 'Indie', 'Massively Multiplay...",2014.0,0.00,0,2015.0,730,True,0
17626,76561198222628548,Dungeon Defenders II,Trendy Entertainment,"['Action', 'Free to Play', 'Indie', 'RPG', 'St...",2017.0,0.00,1222,2015.0,370240,False,0


In [123]:
#Eliminamos las columna innecesarias:

columnas_a_eliminar = ['title', 'developer', 'price', 'posted', 'item_id', 'recommend', 'sentiment_analysis']
df_user_genre.drop(columnas_a_eliminar, axis=1, inplace=True)

In [124]:
df_user_genre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17628 entries, 0 to 17627
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user_id           17628 non-null  object 
 1   genres            17628 non-null  object 
 2   release_date      17628 non-null  float64
 3   playtime_forever  17628 non-null  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 551.0+ KB


In [125]:
# Cambiamos el tipo de dato de la columna release_date a entero
df_user_genre['release_date'] = df_user_genre['release_date'].astype(int)

In [126]:
df_user_genre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17628 entries, 0 to 17627
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   user_id           17628 non-null  object
 1   genres            17628 non-null  object
 2   release_date      17628 non-null  int32 
 3   playtime_forever  17628 non-null  int64 
dtypes: int32(1), int64(1), object(2)
memory usage: 482.1+ KB


Desanidamos la columna genres, asignando un nuevo registro para cada genero:

In [127]:
df_user_genre['genres'] = df_user_genre['genres'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else x)
df_user_genre = df_user_genre.explode('genres')
df_user_genre

,user_id,genres,release_date,playtime_forever
0,76561197970982479,Action,2000,6
1,js41637,Action,2000,0
2,WeiEDKrSat,Action,2000,328
3,jarrodtrim,Action,2000,337
4,UTNerd24,Action,2000,5
...,...,...,...,...
17626,76561198222628548,Indie,2017,1222
17626,76561198222628548,RPG,2017,1222
17626,76561198222628548,Strategy,2017,1222
17627,72947282842,Action,2013,69


Transformamos todos los registros de genres a lowercase para evitar diferencias entre minúscula y mayúscula:

In [128]:
df_user_genre['genres'] = df_user_genre['genres'].str.lower()

In [129]:
df_user_genre

,user_id,genres,release_date,playtime_forever
0,76561197970982479,action,2000,6
1,js41637,action,2000,0
2,WeiEDKrSat,action,2000,328
3,jarrodtrim,action,2000,337
4,UTNerd24,action,2000,5
...,...,...,...,...
17626,76561198222628548,indie,2017,1222
17626,76561198222628548,rpg,2017,1222
17626,76561198222628548,strategy,2017,1222
17627,72947282842,action,2013,69


Guardamos en dataframe en un archivo CSV:

In [130]:
df_user_genre.to_csv(r'C:\Users\Coder\Documents\PI_ML_OPS\Datos\df_user_genre.csv', index=False, errors='replace', encoding='utf-8')

FUNCION END POINT 2

In [131]:
def UserForGenre(genero: str):
    # Filtramos el DataFrame por el género dado
    df_filtered = df_user_genre[df_user_genre['genres'].str.lower() == genero.lower()]

    if df_filtered.empty:
        return {"Mensaje": f"No hay datos para el género {genero}."}

    # Se busca el usuario con más horas jugadas
    usuario_max_horas = df_filtered.loc[df_filtered['playtime_forever'].idxmax()]['user_id']

    # Se crear una lista de acumulación de horas jugadas por año
    acumulacion_por_ano = df_filtered.groupby(df_filtered['release_date'].astype(int).astype(str).str[:4])['playtime_forever'].sum().reset_index()
    horas_por_ano = [{"Año": int(row['release_date']), "Horas": row['playtime_forever']} for index, row in acumulacion_por_ano.iterrows()]

    # Se crear el diccionario de retorno
    resultado = {
        "Usuario con más horas jugadas para Género ": usuario_max_horas,
        "Horas jugadas": horas_por_ano
    }

    return resultado



Comprobamos la función: 

In [132]:

genero = "rpg"  # se indica el genero deseado
resultado = UserForGenre(genero)
print(resultado)

{'Usuario con más horas jugadas para Género ': '76561198166194627', 'Horas jugadas': [{'Año': 1997, 'Horas': 239}, {'Año': 2000, 'Horas': 24}, {'Año': 2002, 'Horas': 18}, {'Año': 2003, 'Horas': 8141}, {'Año': 2005, 'Horas': 52856}, {'Año': 2006, 'Horas': 24546}, {'Año': 2007, 'Horas': 21428}, {'Año': 2008, 'Horas': 24296}, {'Año': 2009, 'Horas': 22546}, {'Año': 2010, 'Horas': 58759}, {'Año': 2011, 'Horas': 1026589}, {'Año': 2012, 'Horas': 238201}, {'Año': 2013, 'Horas': 63509}, {'Año': 2014, 'Horas': 3570}, {'Año': 2015, 'Horas': 141}, {'Año': 2016, 'Horas': 41664}, {'Año': 2017, 'Horas': 5282}]}


__________________________________________________________________________________________________________________

## Dataframe y función para  UsersRecommend (Endpoint 3)
Para UsersRecommend Se debe crear una función que devuelva el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)


Pare ello necesitamos extraer del archivo "mergueFinal" las siguientes columnas:

title
release_date 
sentiment_analysis 

Cargamos los dos datasets mergueFina.parquet:

In [133]:

rutamerge = r'C:\Users\Coder\Documents\PI_ML_OPS\Datos\mergefinal.parquet'
df_users_recommend = pd.read_parquet(rutamerge)

In [134]:
df_users_recommend.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17628 entries, 0 to 17627
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   user_id             17628 non-null  object 
 1   title               17628 non-null  object 
 2   developer           17628 non-null  object 
 3   genres              17628 non-null  object 
 4   release_date        17628 non-null  float64
 5   price               17628 non-null  float64
 6   playtime_forever    17628 non-null  int64  
 7   posted              17628 non-null  float64
 8   item_id             17628 non-null  int64  
 9   recommend           17628 non-null  bool   
 10  sentiment_analysis  17628 non-null  int64  
dtypes: bool(1), float64(3), int64(3), object(4)
memory usage: 1.4+ MB


In [135]:
#Eliminamos las columna innecesarias:
#user_id developer	genres	price	playtime_forever posted	item_id	recommend	

columnas_a_eliminar = ['user_id', 'developer', 'genres', 'price', 'playtime_forever', 'posted', 'item_id', 'recommend']
df_users_recommend = df_users_recommend.drop(columnas_a_eliminar, axis=1)

In [136]:
df_users_recommend.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17628 entries, 0 to 17627
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               17628 non-null  object 
 1   release_date        17628 non-null  float64
 2   sentiment_analysis  17628 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 413.3+ KB


In [137]:
# Cambiamos el tipo de dato de la columna release_date a entero
df_users_recommend['release_date'] = df_users_recommend['release_date'].astype(int)

In [138]:
df_users_recommend.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17628 entries, 0 to 17627
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   title               17628 non-null  object
 1   release_date        17628 non-null  int32 
 2   sentiment_analysis  17628 non-null  int64 
dtypes: int32(1), int64(1), object(1)
memory usage: 344.4+ KB


In [139]:
df_users_recommend

,title,release_date,sentiment_analysis
0,Counter-Strike,2000,2
1,Counter-Strike,2000,2
2,Counter-Strike,2000,1
3,Counter-Strike,2000,2
4,Counter-Strike,2000,1
...,...,...,...
17623,Insurgency,2014,1
17624,Real Horror Stories Ultimate Edition,2014,1
17625,Infinity Wars: Animated Trading Card Game,2014,0
17626,Dungeon Defenders II,2017,0


Guardamos el dataframe en un archivo.csv

In [140]:
df_users_recommend.to_csv(r'C:\Users\Coder\Documents\PI_ML_OPS\Datos\df_users_recommend.csv', index=False, errors='replace', encoding='utf-8')

FUNCION DEL ENDPOINT 3:

In [141]:
def UsersRecommend(df_users_recommend, año):
    # Se realiza un filtrado por año
    df_filtered = df_users_recommend[df_users_recommend['release_date'] == año]

    # Se filtran las recomendaciones positivas/neutrales
    df_filtered = df_filtered[df_filtered['sentiment_analysis'].isin([1, 2])]

    # Se realiza una agrupación por título y con las recomendaciones
    game_recommendations = df_filtered.groupby('title')['sentiment_analysis'].count()

    # Se ordenan de mayor a menor y se tomar los 3 primeros
    top_3_recommendations = game_recommendations.sort_values(ascending=False).head(3)

    # Se crea una la lista con los resultados
    result_list = [{"Puesto {}: {}".format(i + 1, game): count} for i, (game, count) in enumerate(top_3_recommendations.items())]

    return result_list

Prueba de la función_

In [142]:
año_deseado = 2008  # Se ingresa el año que se desea desees
resultados = UsersRecommend(df_users_recommend, año_deseado)
print(resultados)

[{'Puesto 1: Dead Space': 27}, {'Puesto 2: Left 4 Dead': 27}, {'Puesto 3: SPORE™': 21}]


Ésta función sólo funcionará si se ingresa un año que esté dentro del registro el cual está dentro de la siguiente lista:

In [143]:
años_listado = df_users_recommend['release_date'].unique()
años_listado_ordenados = sorted(años_listado, reverse=True)
print(años_listado_ordenados)

[2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002, 2001, 2000, 1999, 1998, 1997, 1995, 1994, 1993, 1990]


__________________________________________________________________________________________________________________

## Dataframe y función para  UsersWorstDeveloper (Endpoint 4)
Para UsersRecommend Se debe crear una función que devuelve el top 3 de desarrolladoras con juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)


Pare ello necesitamos extraer del archivo "mergueFinal" las siguientes columnas:

developer
release_date 
sentiment_analysis 

Para ello cargamos dataframe del archivo mergefinal:

In [144]:
rutamerge = r'C:\Users\Coder\Documents\PI_ML_OPS\Datos\mergefinal.parquet'
df_worst_dev = pd.read_parquet(rutamerge)

In [145]:
df_worst_dev

,user_id,title,developer,genres,release_date,price,playtime_forever,posted,item_id,recommend,sentiment_analysis
0,76561197970982479,Counter-Strike,Valve,['Action'],2000.0,9.99,6,2011.0,1250,True,2
1,js41637,Counter-Strike,Valve,['Action'],2000.0,9.99,0,2014.0,251610,True,2
2,WeiEDKrSat,Counter-Strike,Valve,['Action'],2000.0,9.99,328,2013.0,4000,True,1
3,jarrodtrim,Counter-Strike,Valve,['Action'],2000.0,9.99,337,2012.0,440,True,2
4,UTNerd24,Counter-Strike,Valve,['Action'],2000.0,9.99,5,2014.0,244210,True,1
...,...,...,...,...,...,...,...,...,...,...,...
17623,celebrexISGO,Insurgency,New World Interactive,"['Action', 'Indie', 'Strategy']",2014.0,9.99,0,2015.0,730,True,1
17624,BBiiiirr,Real Horror Stories Ultimate Edition,GameORE,['Indie'],2014.0,1.99,0,2015.0,440,True,1
17625,Hahneex,Infinity Wars: Animated Trading Card Game,Lightmare Studios,"['Free to Play', 'Indie', 'Massively Multiplay...",2014.0,0.00,0,2015.0,730,True,0
17626,76561198222628548,Dungeon Defenders II,Trendy Entertainment,"['Action', 'Free to Play', 'Indie', 'RPG', 'St...",2017.0,0.00,1222,2015.0,370240,False,0


In [146]:
#Eliminamos las columna innecesarias:

columnas_a_eliminar = ['user_id', 'title', 'genres', 'price', 'playtime_forever', 'posted', 'item_id', 'recommend']
df_worst_dev= df_worst_dev.drop(columnas_a_eliminar, axis=1)

In [147]:
df_worst_dev.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17628 entries, 0 to 17627
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   developer           17628 non-null  object 
 1   release_date        17628 non-null  float64
 2   sentiment_analysis  17628 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 413.3+ KB


In [148]:
# Cambiamos el tipo de dato de la columna release_date a entero
df_worst_dev['release_date'] = df_worst_dev['release_date'].astype(int)

In [149]:
df_worst_dev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17628 entries, 0 to 17627
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   developer           17628 non-null  object
 1   release_date        17628 non-null  int32 
 2   sentiment_analysis  17628 non-null  int64 
dtypes: int32(1), int64(1), object(1)
memory usage: 344.4+ KB


In [150]:
df_worst_dev.to_csv(r'C:\Users\Coder\Documents\PI_ML_OPS\Datos\df_worst_dev.csv', index=False, errors='replace', encoding='utf-8')

FUNCIÓN DEL ENDPOINT 4:

In [151]:
def UsersWorstDeveloper(df_worst_dev, año):
    # Filtrar por año
    df_filtered = df_worst_dev[df_worst_dev['release_date'] == año]

    # Filtrar por recomendaciones negativas
    df_filtered = df_filtered[df_filtered['sentiment_analysis'] == 0]

    # Contar la cantidad de juegos no recomendados por cada desarrolladora
    worst_dev_counts = df_filtered['developer'].value_counts()

    # Tomar las 3 desarrolladoras con menos juegos recomendados
    worst_dev_top_3 = worst_dev_counts.tail(3)

    # Crear la lista de resultados
    result_list = [{"Puesto {}: {}".format(i + 1, dev): count} for i, (dev, count) in enumerate(worst_dev_top_3.items())]

    return result_list

Prueba:

In [152]:
año_deseado = 2011  # Se ingresa el año que se desea
resultados_worst_dev = UsersWorstDeveloper(df_worst_dev, año_deseado)
print(resultados_worst_dev)

[{'Puesto 1: 800 North and Digital Ranch': 1}, {'Puesto 2: Volition': 1}, {'Puesto 3: Nadeo': 1}]


__________________________________________________________________________________________________________________

## Dataframe y función para  sentiment_analysis (Endpoint 5)
Para sentiment_analysis, según la empresa desarrolladora, se devuelve un diccionario con el nombre de la desarrolladora como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor.


Pare ello necesitamos extraer del archivo "df_worst_dev.csv" (creado en el desarrollo de la anterior función), las siguientes columnas:


developer
sentiment_analysis 

In [153]:
df_sentiment = pd.read_csv(r'C:\Users\Coder\Documents\PI_ML_OPS\Datos\df_worst_dev.csv')

In [154]:
df_sentiment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17628 entries, 0 to 17627
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   developer           17628 non-null  object
 1   release_date        17628 non-null  int64 
 2   sentiment_analysis  17628 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 413.3+ KB


Eliminamos la columma release_date:

In [155]:
df_sentiment = df_sentiment.drop('release_date', axis=1)

In [156]:
df_sentiment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17628 entries, 0 to 17627
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   developer           17628 non-null  object
 1   sentiment_analysis  17628 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 275.6+ KB


Guardamos el dataframe en un archivo csv:

In [157]:
df_sentiment.to_csv(r'C:\Users\Coder\Documents\PI_ML_OPS\Datos\df_sentiment.csv', index=False, errors='replace', encoding='utf-8')

FUNCIÓN DEL ENDPOINT 5:

In [158]:
def sentiment_analysis(developer):
    # Se filtra el DataFrame para obtener solo las filas que corresponden al developer
    df_empresa = df_sentiment[df_sentiment['developer'] == developer]
    
    # Se cuentan la cantidad de registros de cada categoría de sentiment_analysis
    sentiment_counts = df_empresa['sentiment_analysis'].value_counts().to_dict()
    
    # Se crea el diccionario retorno
    result_dict = {developer: sentiment_counts}
    
    return result_dict


Realizamos una prueba de la función:

In [159]:
nombre_developer = 'Rockstar Games'  # Se ingresa el nombre del developer
resultado = sentiment_analysis(nombre_developer)

print(resultado)

{'Rockstar Games': {2: 14, 1: 9, 0: 2}}


La función anterior sólo mostrará los resultados de los nombres de las empresas desarrolladoras que se encuentran en la columna developer:

In [160]:
print("Developers registrados:")
for nombre in df_sentiment['developer'].unique():
    print(nombre)

Developers registrados:
Valve
Tripwire Interactive
Gearbox Software
Facepunch Studios
Firaxis Games
Ubisoft Montreal
Streum On Studio
Re-Logic
PopCap Games, Inc.
Avalanche Studios
Outerlight Ltd.
Rockstar Games
EA Redwood Shores
Black Element
EA Los Angeles
OVERKILL Software
kChamp Games
Reality Pump Studios
The Creative Assembly,Feral Interactive (Mac),Feral Interactive (Linux)
Obsidian Entertainment
Square Enix
Elecorn
Reloaded Productions
Firaxis Games,Aspyr (Mac, Linux)
The Creative Assembly,Feral Interactive (Mac)
Criterion Games
Infinity Ward,Aspyr (Mac)
Trek Industries, Inc
The Creative Assembly
Io-Interactive A/S
Crytek
Infinity Ward
Maxis™
Oddworld Inhabitants
Cryptic Studios
Introversion Software
Bethesda Game Studios
Double Fine Productions
Ion Storm
Troika Games
Frozenbyte
Relic Entertainment,Feral Interactive (Mac/Linux)
Volition
Relic Entertainment
Strategy First
Arkane Studios
MinMax Games Ltd.
Crystal Dynamics
Frontier,Aspyr (Mac)
Triumph Studios,Virtual Programming
GSC